In [1]:
import io

import cv2 as cv
import ipywidgets as widgets
import numpy as np
from PIL import Image, ImageDraw

CASCADE = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

# WIDGETS 
uploader = widgets.FileUpload(button_style='info')
scale_slider = widgets.FloatSlider(
    value=1.05, 
    min=1.05,  # Must be >=1.01
    max=1.50,
    step=0.05,
    description='Scale Factor',
    continuous_update=False,  # Debounce for slider handler
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
neighbor_slider = widgets.IntSlider(
    value=3,  # Set default slider value to default minNeighbors argument
    min=1,
    max=10,
    step=1,
    description='Neighbors',
    continuous_update=False,  # Debounce for slider handler
    orientation='horizontal',
    readout=True)
widget_output = widgets.Output()

# DETECTION FUNCTIONS 
def get_upload_formats():
    """Return PIL and grayscale formats from uploaded image."""
    img_bytestring = uploader.data[0]  # Get bytestring from uploader widget
    pil_img = Image.open(io.BytesIO(img_bytestring))  # Convert bytestring to PIL image
    np_array = np.frombuffer(img_bytestring, dtype=np.uint8)  # Convert bytestring to 1-D numpy array
    grayscale = cv.imdecode(np_array, cv.IMREAD_GRAYSCALE)  # Convert 1-D np array to 2-D numpy array
    formats = {"pil_img": pil_img, "grayscale": grayscale}
    return formats


def draw_faces(scale_factor, min_neighbors):
    """Return copy of image with boxes drawn around detected faces"""
    formats = get_upload_formats()
    detected_faces = CASCADE.detectMultiScale(formats['grayscale'],
                                              scale_factor, min_neighbors)
    
    img_copy = formats['pil_img'].copy() # Make copy of original image for drawing
    draw_context = ImageDraw.Draw(img_copy) # Set up drawing context

    # Use list of detected faces to draw boxes on img_copy, return
    for face in detected_faces:
        draw_context.rectangle((face[0],face[1],face[0]+face[2],face[1]+face[3]), outline='red', width=2)
    return img_copy 

# HANDLER FUNCTIONS FOR WIDGETS
def handle_scale_slider(change):
    with widget_output:
        try: 
            display(draw_faces(change['new'], neighbor_slider.value))
        except IndexError: # Handle slider movement w/o upload
            display('Please upload an image first.')
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed

    
def handle_neighbor_slider(change):
    with widget_output:
        try:      
            display(draw_faces(scale_slider.value, change['new']))
        except IndexError: # Handle slider movement w/o upload
            display('Please upload an image first.')
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed

    
def handle_upload(change):
    # Reset values after >1 upload
    scale_slider.value = 1.05
    neighbor_slider.value = 3
    
    with widget_output:
        display(draw_faces(scale_slider.value, neighbor_slider.value))
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed
      
### OBSERVERS (LINKS WIDGETS TO HANDLER FUNCTIONS)
scale_slider.observe(handle_scale_slider, names='value')
neighbor_slider.observe(handle_neighbor_slider, names='value')
uploader.observe(handle_upload, names='_counter')

### DISPLAY WIDGETS, OUTPUT
widgets.VBox([widgets.Label('Upload an image.'),
              uploader, scale_slider, neighbor_slider, widget_output])